In [8]:
%load_ext autoreload
import datetime
# import ete3
# import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import xml.etree.ElementTree as ET
import xml.dom.minidom
# import xmlschema

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
# from maxes.print_xes_info import PrintXesInfo
import maxes.analyze_sequence
import maxes.serialization.serialize
# import maxes.graphs
# import maxes.utils
from maxes.generators.xes_generator1 import XesGenerator1

In [10]:
import logging
import importlib

def setup_logging():
    # TODO: use __vsc_ipynb_file__
    importlib.reload(logging)
    filename_timestamp_part = datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    filename = f"C:\\vt\\md\\maxes\\output\\logs\\notebooks\\{filename_timestamp_part}.log"
    logging.basicConfig(
        filename=filename,
        filemode="a",
        format="%(asctime)s %(message)s",
        level=logging.DEBUG)
    logging.info("START")

setup_logging()

In [20]:
def generate_xes_for_file(source_file_path, destination_file_path):
    loader = XesLoader({ "drop_timezones": True })

    logging.info("Loading")
    log = loader.load(source_file_path)

    if len(loader.errors):
        raise RuntimeError("Errors while loading XES")

    logging.info("Creating dataframes")
    for trace in log.traces:
        trace.update_df()

    logging.info("Fitting")
    generator = XesGenerator1(debug=True).fit(log)

    logging.info("Generating")
    generated_log = generator.generate()

    logging.info("Serializing")
    generated_log_ET = maxes.serialization.serialize.Serializer().serialize(generated_log, xml_log_skeleton=log.loader.xml_log_skeleton)

    logging.info("Formatting XML")
    ET.indent(generated_log_ET)

    logging.info("Writing XML")
    ET.register_namespace("", "http://www.xes-standard.org")
    with open(destination_file_path, "w") as file:
        generated_log_ET.write(file, encoding="unicode")

In [21]:
generate_xes_for_file(
    source_file_path="C:\\vt\\md\\maxes\\data\\JUnit 4.12 Software Event Log_1_all\\JUnit 4.12 Software Event Log.xes\\JUnit 4.12 Software Event Log.xes",
    destination_file_path="C:\\vt\\md\\maxes\\output\\generated\\junit.xes"
)